In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/mmkshira/spark-3.2.0-bin-hadoop2.7"

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")
# sc = SparkContext("local", "test-app")
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

from pyspark.sql import SparkSession
import random

spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

### importing libraries
#json
# import jsonlines
#spark
from pyspark.sql.functions import explode,col
from pyspark.sql import functions as F
from pyspark.sql.functions import concat_ws
#plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# matplotlib and wordcloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#dash
from jupyter_dash import JupyterDash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output 
# numerical computing
import pandas as pd
import numpy as np

21/12/10 22:27:31 WARN Utils: Your hostname, scslt388 resolves to a loopback address: 127.0.1.1; using 192.168.2.19 instead (on interface wlp0s20f3)
21/12/10 22:27:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 22:27:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/10 22:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/10 22:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/10 22:27:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
data = spark.read.json("sample.jsonl").sort("year",ascending=False)


21/12/10 22:27:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
summary = data.select("id","papertitle","authors","year","citationcount","topics","language")

In [5]:
summary.show()

+----------+--------------------+--------------------+----------+-------------+--------------------+
|        id|          papertitle|             authors|      year|citationcount|              topics|
+----------+--------------------+--------------------+----------+-------------+--------------------+
|2960426479|secure distribute...|[{University of S...|2020-01-01|            3|[computer vision,...|
|2930137557|a poor man s appr...|[{Technical Unive...|2020-01-01|            2|[finite element m...|
|2919403121|filtering techniq...|[{Northumbria Uni...|2020-01-01|            2|[data mining, fea...|
|2996755977|sign opt a query ...|[{IBM (United Sta...|2020-01-01|            3|[directional deri...|
|2954810543|a real time image...|[{Jiangnan Univer...|2020-01-01|            1|[kernel, computer...|
|2946528256|discrete multiobj...|[{Malaviya Nation...|2020-01-01|            1|[telecommunicatio...|
|2996197041|emerging frontier...|[{Philips (Finlan...|2020-01-01|            1|[knowledge m

In [14]:
n_papers = summary.count()
n_languages = summary.select("language").distinct().count()
n_authors = summary.select(explode("authors").alias("author"))\
        .select(F.col("author").getItem("id").alias("author_id")).distinct().count()
n_topics = summary.select(explode("topics").alias("topic"))\
        .select("topic").distinct().count()
n_institutes = summary.select(explode("authors").alias("author"))\
        .select(F.col("author").getItem("affiliation").alias("affiliation")).distinct().count()


print(n_papers)
print(n_authors)
print(n_topics)
print(n_institutes)
print(n_languages)

50000
142140
33478
3942
109
